# 문제
1. corona.csv 파일 로드
2. Unnamed: 0 컬럼을 삭제
3. 컬럼을 첫번째부터 세번째까지 컬럼만 두고 전부 삭제
4. 컬럼의 이름 변경 ['등록일시', '누적사망자', '누적확진자']
5. 등록일시를 기준으로 오름차순 정렬
6. 일일확진자와 일일사망자 파생변수 생성(결측치는 0으로 대체)
7. 일일확진자가 음수인 값을 삭제
8. 등록일시를 시계열로 변경하고 인덱스로 변경
9. 요일이라는 파생변수를 만들어서 요일 대입
10. 요일 컬럼 0 -> 월, 1 -> 화, .... 6 -> 일로 변경
11. 요일별 일일 확진자의 평균을 바 그래프로 표시
12. 요일별 일일 사망자의 평균을 바 그래프로 표시

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('../csv/corona.csv', usecols=[1, 2, 3])

In [6]:
df.columns = ['등록일시', '누적사망자', '누적확진자']

In [10]:
df.sort_values('등록일시', ascending=True, inplace=True)

In [15]:
df['일일확진자'] = df['누적확진자'].diff().fillna(0)
df['일일사망자'] = (df['누적사망자'] - df['누적사망자'].shift()).fillna(0)
df

,등록일시,누적사망자,누적확진자,일일확진자,일일사망자
819,2020-03-10 00:00:00.000,54,7513,0.0,0.0
818,2020-03-11 00:00:00.000,60,7755,242.0,6.0
817,2020-03-12 00:00:00.000,66,7869,114.0,6.0
816,2020-03-13 00:00:00.000,67,7979,110.0,1.0
815,2020-03-14 00:00:00.000,72,8086,107.0,5.0
...,...,...,...,...,...
4,2022-06-04 08:56:49.219,24238,18153814,12039.0,9.0
3,2022-06-05 08:53:19.426,24258,18163648,9834.0,20.0
2,2022-06-06 09:00:06.734,24279,18168670,5022.0,21.0
1,2022-06-07 09:09:00.897,24299,18174842,6172.0,20.0


In [18]:
df = df.loc[~(df['일일확진자'] < 0)]

In [22]:
df['등록일시'] = pd.to_datetime(df['등록일시'])

/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_13598/1929878470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록일시'] = pd.to_datetime(df['등록일시'])


In [24]:
df.set_index('등록일시', inplace=True)

In [25]:
df2 = df.copy()

In [26]:
for i in df.index:
    df.loc[i, '요일'] = i.weekday()

/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_13598/3135090362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, '요일'] = i.weekday()


In [28]:
_list = df['요일'].astype(int).unique()

In [29]:
week_name = ['월', '화', '수', '목', '금', '토', '일']

for i in _list:
    df.loc[df['요일'] == i, '요일'] = week_name[i]

In [30]:
df['요일'].value_counts()

요일
화    118
수    118
금    117
토    117
일    117
목    116
월    116
Name: count, dtype: int64

In [31]:
import numpy as np

In [34]:
import platform
import matplotlib.pyplot as plt

if platform.system() == 'Darwin':
    plt.rc('font', family = 'AppleGothic')
else:
    plt.rc('font', family = 'MalgunGothic')

<Axes: xlabel='요일'>

In [36]:
df.groupby('요일')['일일확진자'].mean().plot.bar()

<Axes: xlabel='요일'>

In [37]:
df.groupby('요일')['일일사망자'].mean().plot.bar()